# 데코레이터와 DRY 원칙
데코레이터를 사용하여 특정 로직을 분리된 컴포넌트로 추상화하면 여러 객체에 데코레이터를 적용하여 코드의 재사용이 용이해짐  
**: DRY(Don't Repeat Yourself) 원칙을 잘 따르게 됨**

코드 재사용을 위해 데코레이터를 사용할 때 염두에 두어야 할 것은 실질적으로 코드 사용량을 줄일 수 있다는 확실한 믿음이 있어야 한다는 것  
- 모든 데코레이터(특히 신중하게 설계되지 않은 데코레이터)는 코드의 복잡성을 증가시키므로 복잡성이 가치가 있어야 함
- 그다지 재사용할 필요가 없을 경우에는 별개의 함수나 작은 클래스로도 충분한 경우가 있음

데코레이터 사용을 권하는 경우
- 처음부터 데코레이터를 만들지 말고 패턴이 생기고 데코레이터에 대한 추상화가 명확해지면 그 때 리팩토링을 진행  
- 데코레이터가 적어도 3회 이상 필요한 경우에만 구현  
- 데코레이터 코드를 최소한으로 유지

# 데코레이터와 관심사의 분리  
코드 재사용의 핵심은 응집력 있는 컴포넌트를 만드는 것  
최소한의 책임을 가져서 오직 한 가지 일만 해야 하며, 그 일을 잘 해야 함  
컴포넌트가 작을 수록 재사용성이 높아짐  

아래 예시는 특정 함수의 실행을 추적하는 데코레이터

In [1]:
from functools import wraps

def traced_function(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("%s 함수 실행", function.__qualname__)
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "함수 %s의 처리 소요시간 %.2fs", 
            function.__qualname__, 
            time.time() - start_time
        )
        return result
    return wrapped

위 데코레이터는 동작에 문제가 있는데, 하나 이상의 작업을 수행하고 있기 때문임  
특정 함수가 방금 호출된 것을 기록하고 실행하는데 걸린 시간도 기록함.  
오직 한 가지만 원하는 경우에도 두 가지 책임을 실행하고 있는 것 

따라서 위의 예시는 더 구체적이고 제한적인 책임을 지닌 더 작은 데코레이터로 분류되어야 함.

In [2]:
def log_execution(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function.__qualname__)
        return function(*args, **kwargs)
    return wrapped

def measure_time(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs", 
            function.__qualname__, 
            time.time() - start_time
        )
        return result
    return wrapped

위의 예시처럼 분리된 데코레이터는 다음과 같이 조합하여 두가지의 기능을 모두 사용할 수 있음
```python
@measure_time
@log_execution
def operation():
    ...
```

# 좋은 데코레이터 분석
### 훌륭한 데코레이터가 갖추어야 할 특성
1. 캡슐화와 관심사의 분리: 좋은 데코레이터는 실제로 하는 일과 데코레이팅 하는 일의 책임을 명확히 구분해야 함. 데코레이터의 클라이언트는 내부에서 어떻게 구현했는지 전혀 알 수 없는 블랙박스 모드로 동작해야 함  
2. 독립성: 데코레이터가 하는 일은 독립적이어야 하며, 데코레이팅 되는 객체와 최대한 분리되어야 함
3. 재사용성: 데코레이터는 하나의 함수 인스턴스에만 적용되는 것이 아니라 여러 유형에 적용 가능한 형태가 바람직함. 하나의 함수에만 적용된다면 데코레이터가 아니라 함수로 대신할 수도 있기 때문에 데코레이터는 충분히 범용적이어야함  

좋은 데코레이터 예시는 Celery 프로젝트에서 확인 가능  
Celery 프로젝트에서는 app.task 데코레이터를 함수에 적용하여 작업을 정의  
```python
@app.task
def mytask():
    ...
```
이것이 좋은 데코레이터인 이유 중 하나는 캡슐화가 매우 잘 되어 있기 때문임.  
라이브러리 사용자는 함수 본문을 정의하기만 하면 데코레이터가 이를 자동으로 작업으로 변환  
app.task 데코레이터는 많은 로직과 코드를 래핑하지만 아무것도 수행하는 함수와 관련이 없음 = **완벽한 캡슐화 & 관심사의 분리**  

데코레이터의 또 다른 일반적인 사용 예는 웹 프레임워크 -> 뷰 핸들러는 데코레이터를 통해 URL로 등록
```python
@route("/", method=["GET"])
def view_handler(request):
    ...
```
웹프레임워크의 사용자는 @route가 데코레이터가 하는 일을 거의 알지 못하므로 완벽한 캡슐화를 제공  

**좋은 데코레이터는 깔끔한 인터페이스를 제공하고 사용자가 내부 동작 원리를 자세히 몰라도 기대하는 바를 이룰 수 있게 해줌**

# 정리 / 요약
파이썬의 데코레이터: 클래스, 메서드, 함수, 제너레이터 등에 적용 가능  

#### 함수를 위한 데코레이터
- 원래 함수의 시그니처와 일치하도록 만듦
- 일반 &#42;args 나 &#42;&#42;kwargs를 사용하는 대신 서명을 원본과 일치시키면 원본 함수와 유사하므로 사용자에게 더 익숙해지고 가독성과 유지보수성이 개선됨. 

#### 코드 재사용 및 DRY 원칙
- 데코레이터는 코드를 재사용하고 DRY 원칙을 따르는 데 매우 유용한 도구
- 그러나 현명하게 사용하지 않으면 복잡성이 이익보다 늘 수 있으므로 3번 이상 여러번 적용될 떄만 데코레이터 사용하기
- 관심사의 분리를 위해 가능한 작게 데코레이터 유지하기  

#### 깔끔한 인터페이스
- 로직의 일부를 데코레이터로 추출하여 클래스의 정의를 단순화할 수 있음  
- 데코레이터는 사용자에게 내부 작동 원리를 숨기면서도 어떤 일을 하는 지 알려줌으로써 가독성을 높여줌